In this script, we select only the cities that contain one or more dams to reduce the complexity of the analysis that will follow and export the list as a CSV.

In [1]:
import geobr
import geopandas as gpd
import numpy as np
import osmnx as ox
import pandas as pd
from unidecode import unidecode
from shapely.geometry import Point, Polygon
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')

#### General helper functions

In [2]:
def read_dams():
    '''
    Reads the dam safety dataset using
    the appropriate configurations.
    '''

    dams = pd.read_csv("../../data/brazil/snisb/dam-report-07022021.csv", encoding='Latin5', sep=';', skiprows=[0,1])
    return dams

In [3]:
def make_gdf(df):
    '''
    Converts the dataframe to a geodaframe
    using the columns Longitude and Latitude
    ---
    Parameters:
    
    df -> The dam safety dataframe
    '''
    
    df['geometry'] = df.apply(lambda row: Point(row.longitude, row.latitude), axis=1)
    
    df = gpd.GeoDataFrame(df)
    
    return df

In [4]:
def crs_to_area(gdf):
    '''
    Converts the CRS for equal
    area calculations.
    ---
    Parameters:
    
    gdf -> A geodataframe
    '''
    
    return gdf.to_crs('''PROJCS["Brasil_Albers_Equal_Area",GEOGCS["GCS_WGS_1984",DATUM["D_SIRGAS_2000",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers"],PARAMETER["false_easting",5000000.0],PARAMETER["false_northing",10000000.0],PARAMETER["central_meridian",-54.0],PARAMETER["standard_parallel_1",-2.0],PARAMETER["standard_parallel_2",-22.0],PARAMETER["latitude_of_origin",-12.0],UNIT["Meter",1.0]]''')

In [5]:
def crs_to_coords(gdf):
    '''
    Converts the CRS of the geodataframe
    to the Brazilian standard for geogra-
    phic projections.
    ---
    Parameters:
    
    gdf -> A geodataframe
    '''
    
    return gdf.to_crs("EPSG:4674")

In [6]:
def create_buffer(gdf, r):
    '''
    Creates a buffer around the
    geometries of the given gdf
    with a r radius.
    '''
    
    gdf.geometry = gdf.geometry.buffer(r)
    
    return gdf

#### Data cleaning functions
The Brazilian National Dam Safety Information System data is a mess. Let's clean it up a bit.

In [7]:
def standardize_columns(df):
    '''
    Remove special characters from the column
    names and makes them all lowercase. 
    ---
    Parameters:
    
    df -> The dam safety dataframe
    '''
    
    df.columns = df.columns.map(unidecode)
    df.columns = df.columns.map(lambda x: x.lower())
    df.columns = df.columns.map(lambda x: x.strip())
    
    return df

In [8]:
def fix_separators(df):
    '''
    The Latitude and Longitude columns in the dataframe
    are currently stored as strings with a ',' as the decimal
    separator. This function changes the separator to '.' and
    casts it to float.
    ---
    Parameters:
    
    df -> The dam safety dataframe
    '''
    
    df.latitude = df.latitude.str.replace(",", ".").astype(float)
    df.longitude = df.longitude.str.replace(",", ".").astype(float)
    
    return df

#### Dam classification functions
Let's mark the dams that we consider specially dangerous – that is, those that have both a high structural risk and a high potential damage.

In [9]:
def classify_risky_dams(df):
    '''
    Marks the dams that have both a
    high risk category and a high
    potential damage so we can proceed 
    in the analysis.
    ---
    Parameters:
    
    df -> The dam safety dataframe
    '''
    
    condition = (df.categoria_de_risco == 'Alto') & (df.dano_potencial_associado == 'Alto')
    
    df['high_risk_high_damage'] = np.where(condition, True, False)
    
    return df

#### Execution

In [10]:
def get_dams():
    '''
    Runs all the functions to prepare
    the dam safety dataset sequentially.
    '''
    
    dams = read_dams()
    dams = standardize_columns(dams)
    dams = fix_separators(dams)
    dams = make_gdf(dams)
    dams = classify_risky_dams(dams)
    dams = dams[dams.high_risk_high_damage] # selects using boolean mask
    dams = dams.set_crs("EPSG:4674") # Brazilian standard projection
    dams = crs_to_area(dams)
    dams = create_buffer(dams, 1000)
    
    return dams

In [11]:
def get_cities_of_interest(dams):
    '''
    Runs all the functions to download and
    select the cities of interest.
    ---
    Parameters:
    
    dams -> the dataframe containing information
    about dams in Brazil
    '''
    
    # Downloads and saves city data
    cities = geobr.read_municipality(code_muni="all", year=2020)
    cities = cities.to_crs(dams.crs)
    cities.to_feather("../../data/brazil/cities/cities.feather")
    
    # Selects the cities that intersect with the radiuses
    cities_of_interest = gpd.sjoin(cities, dams, how='inner', op='intersects')
    
    # Keeps only the id columns and only the unique entries
    cities_of_interest = cities_of_interest.drop_duplicates(subset='code_muni')
    
    # Removes the information associated with the dams
    cities_of_interest = cities_of_interest.drop(columns=['index_right', 'codigo_snisb',
       'nome_da_barragem', 'nome_secundario', 'uso_principal', 'uf',
       'municipio', 'categoria_de_risco', 'dano_potencial_associado',
       'nome_do_empreendedor', 'orgao_fiscalizador',
       'codigo_barragem_fiscalizador', 'regulada_pela_pnsb',
       'numero_da_autorizacao', 'possui_pae', 'possui_plano_de_seguranassa',
       'possui_revisao_periodica', 'data_da_ultima_fiscalizacao',
       'barragem_autuada', 'altura_fundacao_m', 'altura_terreno_m',
       'capacidade_hm3', 'comprimento_coroamento_m', 'tipo_de_material',
       'uso_complementar', 'classe_de_residuo', 'curso_dagua_barrado',
       'regiao_hidrografica', 'unidade_de_gestao', 'dominio',
       'data_da_ultima_inspecao', 'tipo_da_ultima_inspecao',
       'nivel_de_perigo_global', 'possui_eclusa', 'fase_da_vida', 'latitude',
       'longitude', 'completude', 'high_risk_high_damage'])
    
    # Saves file
    cities_of_interest.to_feather("../../data/brazil/cities/cities-of-interest.feather")
    
    return cities_of_interest

In [12]:
def get_grid_of_interest(cities_of_interest):
    '''
    Uses r-trees and spatial indexes to efficiently
    detect which squares in the statistical grid 
    intersect with our area of interest.
    ---
    Parameters:
    
    cities_of_interest -> The dataframe containing the cities of interest
    '''
    
    # Reads data
    grid = gpd.read_feather("../../data/brazil/grade/feather/concatenated/populated.feather")
    
    # Converts CRS
    grid = crs_to_area(grid)
    
    # Creates a polygon which is the union of the cities of interest
    union = cities_of_interest.geometry.unary_union
    
    # Cut the union in many smaller bounding boxes
    bboxes = ox.utils_geo._quadrat_cut_geometry(union, quadrat_width=100000) # 100km side for each box
    
    # Create a spatial index for the grid
    sindex = grid.sindex
    
    # Find the squares that intersect with each bounding box and save their ids
    grid_of_interest = pd.DataFrame()
    for index, bbox in enumerate(bboxes):

        print(f"{index + 1} of {len(bboxes)}", end='\r')

        # Find approximate matches with r-tree, then precise matches from those approximate ones
        possible_matches_index = list(sindex.intersection(bbox.bounds))
        possible_matches = grid.iloc[possible_matches_index]

        precise_matches = possible_matches[possible_matches.intersects(bbox)]
        grid_of_interest = grid_of_interest.append(precise_matches)
        
    # Keeps columns of interest
    grid_of_interest = grid_of_interest[['ID_UNICO', 'QUADRANTE', 'MASC', 'FEM', 'POP',
       'DOM_OCU', 'geometry']]
    
    # No duplicates
    grid_of_interest = grid_of_interest.drop_duplicates(subset='ID_UNICO')
    
    # Saves
    grid_of_interest.to_feather("../../data/brazil/grade/feather/concatenated/grid-of-interest.feather")
    
    return grid_of_interest

In [13]:
def get_tracts_of_interest(grid_of_interest):
    '''
    Runs all the functions to read and
    select the tracts of interest.
    ---
    Parameters:
    
    grid_of_interest -> The dataframe
    containing the squares of interest
    '''
    # Reads tract data and keeps only those that intersect with the cities
    tracts = gpd.read_feather("../../data/brazil/censo/combined/combined.feather")
    tracts = tracts.to_crs(grid_of_interest.crs)
    
    # A zero side buffer is needed to correct self-intercepting geometries
    tracts.geometry = tracts.geometry.buffer(0) 
    
    # Spatial join to keep only tracts in the cities of interest
    tracts_of_interest = gpd.sjoin(tracts, grid_of_interest, how='inner', op='intersects')
    
    # There is some overlap on neighboring tracts (probably from simplified geometries) that we must drop
    tracts_of_interest = tracts_of_interest.drop_duplicates(subset='code_tract')
    
    # Keeps only columns that matter
    tracts_of_interest = tracts_of_interest[['code_tract', 'geometry', 'total_permanent_households',
       'permanent_household_nominal_mean_income', 'total_private_households',
       'private_households_under_minimum_wage', 'total_residents',
       'white_residents', 'black_residents', 'yellow_residents',
       'pardo_residents', 'indigenous_residents', 'literate_residents',
       'pct_private_households_under_minimum_wage', 'pct_literate_residents',
       'pct_black_and_pardo_residents']]
    
    # Saves
    tracts_of_interest.to_feather("../../data/brazil/censo/combined/tracts-of-interest.feather")

    return tracts_of_interest

In [14]:
def main():
    print("Processing dams")
    dams = get_dams()
    
    print("Processing cities of interest")
    cities_of_interest = get_cities_of_interest(dams)
    
    print("Processing grid of interest")
    grid_of_interest = get_grid_of_interest(cities_of_interest)
    
    print("Processing tracts of interest")
    tracts_of_interest = get_tracts_of_interest(grid_of_interest)


In [15]:
%%time
if __name__ == "__main__":
    main()

Processing dams
Processing cities of interest
Processing grid of interest
Processing tracts of interest
CPU times: user 2min 35s, sys: 10.9 s, total: 2min 46s
Wall time: 2min 52s
